In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
sys.path.append(os.path.join(os.path.pardir, 'lesview'))
sys.path.append(os.path.join(os.path.pardir, 'gotmtool'))
from gotmtool import dat_dump_pfl
from lesview import *

In [ ]:
casename = 'lsc_ymc22_sbl_bbl_v2_rf'
datapath = os.path.join(os.path.pardir, 'tests', '{:s}'.format(casename))
filepath = os.path.join(datapath, 'averages.jld2')
data_pfl = OceananigansDataProfile(filepath=filepath)

out_dir = os.path.join(os.path.pardir, 'data', 'gotm', casename)
os.makedirs(out_dir, exist_ok=True)

In [ ]:
ds = data_pfl.dataset
ds

In [ ]:
# convert b to T
g = 9.81
N2 = 1.962e-4
T0 = 20.0
S0 = 35.0
alphaT = 2.0e-4
betaS = 8.0e-4
H = 30
dTdz = N2/alphaT/g
dTdz

In [ ]:
da = ds.data_vars['b']/alphaT/g + T0 - dTdz*H
darr = da.data.transpose()
time = pd.to_datetime(da.time.data)
z = da.z.data
dat_dump_pfl(time, z, [darr], os.path.join(out_dir, 't_prof.dat'), order=1)
da.plot()

In [ ]:
dax = ds.data_vars['u']
day = ds.data_vars['v']
darrx = dax.data.transpose()
darry = day.data.transpose()
time = pd.to_datetime(dax.time.data)
z = dax.z.data

# remove Stokes drift
depth = 30.0 # m
g = 9.81 # m/s2
amplitude = 1.13 # m
wavelength = 60. # m
wavenumber = 2. * np.pi / wavelength # 1/m
frequency = np.sqrt(g * wavenumber * np.tanh(wavenumber * depth)) # 1/s
us0 = amplitude**2 * wavenumber * frequency # m/s

us = us0 * np.cosh(2. * wavenumber * (z + depth)) / (2. * np.sinh(wavenumber * depth)**2)

for i in np.arange(0,time.size):
    darrx[i,:] -= us

dat_dump_pfl(time, z, [darrx, darry], os.path.join(out_dir, 'u_prof.dat'), order=1)
plt.figure()
dax.plot()
plt.figure()
day.plot()

In [ ]:
dax[:,-200:].mean(dim='time').plot(y='z')
plt.plot(np.mean(darrx[-200:,:], axis=0), z)

In [ ]:
dax[:,0].plot(y='z')
plt.plot(darrx[0,:], z)

In [ ]:
da[:,0].plot(y='z')
plt.plot(darr[0,:], z)